# ニューラルネットワークの動作確認（part 4：参考ケース）

<a href="http://scikit-learn.org/dev/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn-neural-network-mlpclassifier"><b>sklearn.neural_network.MLPClassifier</b></a> を使用します。

グリッドサーチにより、条件を変えてみて動きを確認してみます。

この例では、グリッドサーチ内のクロスバリデーションにおいて、プロダクションと同じ<b>__accuracy_score 関数を使用</b>しております。

この結果、<b><a href="05.ipynb">デフォルトのグリッドサーチによる結果</a></b>とは、選択されたパラメータが異なってしまうことが確認されました。

## (1) テストデータ／環境準備

In [1]:
'''
    テスト環境を準備するためのモジュールを使用します。
'''
import sys
import os
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

from prototype.modules import TestTool

### (1-1) テストデータをコピー

In [2]:
'''
    データファイルは、既存の訓練データを別場所にコピーしてから使用します
    テストデータは、csv_file_name で指定した複数件のファイルを使用します。
'''
csv_file_names = [
    #'test_daikin_conversation.csv',
    #'test_benefitone_conversation.csv',
    'test_septeni_conversation.csv',
    #'test_ptna_conversation.csv',
]
temp_path = TestTool.copy_testdata_csv(learning_dir, csv_file_names)

CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_septeni_conversation.csv]


## (2) グリッドサーチ

学習(GridSearchCV.fit)--->評価(Evaluator.evaluate)の流れで実行させ、実行時間計測、およびaccuracy計測を行います。

In [3]:
from time import time
import numpy as np

from sklearn.grid_search import GridSearchCV
from sklearn.neural_network import MLPClassifier

from learning.core.evaluator import Evaluator

def fit_and_cross_validation(path):
    '''
        訓練データのTF-IDFベクターを作成
    '''
    basename = os.path.basename(path)
    print("prepare_tf_idf_vectors: dataset=%s..." % basename)
    t0 = time()

    X, y, vectorizer = TestTool.prepare_tf_idf_vectors(path)
    print("prepare_tf_idf_vectors: done in %0.3fs." % (time() - t0))

    '''
        訓練データ全体を使用して学習実施
        プロダクションと同様、ハイパーパラメータの選択のために
        グリッドサーチを使用します
        
        ハイパーパラメータとして、レイヤー／ユニット数を採用
            レイヤー数：デフォルトの1層から、4層まで変化させます
            ユニット数：デフォルト100／その半分50／その倍200の３通り
    '''
    print("GridSearchCV: search and fitting...")
    t0 = time()

    params = {
        'hidden_layer_sizes': [
            (50,),(100,),(200,), # 1層
            (50,50,),(100,100,),(200,200,), # 2層
            (50,50,50,),(100,100,100,),(200,200,200,), # 3層
            (50,50,50,50,),(100,100,100,100,),(200,200,200,200,), # 4層
        ]
    }

    grid = GridSearchCV(
        MLPClassifier(activation='logistic', max_iter=10000, shuffle=False, random_state=0), 
        param_grid=params,
        scoring=__accuracy_score
    )
    grid.fit(X, y)
    estimator = grid.best_estimator_
    print("GridSearchCV: done in %0.3fs." % (time() - t0))

    ''' 
        クロスバリデーション（モデル評価フェーズ）を実施
        プロダクションと同様、Evaluator クラスを使用して評価します
        
        Evaluator クラスで使用している cross_val_score 関数は、
        引数の estimator により、
        内部で fit, predict, predict_proba の各関数を実行しています。
    '''
    print("Evaluator: evaluating...")
    t0 = time()

    evaluator = Evaluator()
    evaluator.evaluate(estimator, X, y, threshold=0.5)
    print("Evaluator: done in %0.3fs." % (time() - t0))
    
    return (basename, X, y, vectorizer, estimator, evaluator)

def __accuracy_score(estimator, X, y):
    '''
        グリッドサーチ用のスコアリング関数。
        
        プロダクションの Evaluator クラス内で使用しているものと、
        同じものを採用して評価します。
        （すなわちプロダクション・コードからそのまま拝借・・・）
    '''
    y_pred = estimator.predict(X)
    probabilities = estimator.predict_proba(X)
    max_probabilities = np.max(probabilities, axis=1)

    # 予測結果と実際を比較
    bools = y == y_pred
    # しきい値を超えているもののみをTrueにする
    bools = bools == (max_probabilities > 0.5)
    # 正当率を算出
    score = np.sum(bools) / np.size(bools, axis=0)

    '''
        ここだけカスタマイズしています。ご容赦ください
    '''
    params = estimator.get_params()
    print("__accuracy_score: hidden_layer_sizes=%s, accuracy=%0.6f (iterated=%d, loss=%0.6f)" % (
        str(params['hidden_layer_sizes']), 
        score, 
        estimator.n_iter_, 
        estimator.loss_
    ))

    return score


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


### (2-1) グリッドサーチ実行

同じレイヤー／ユニット数の組み合わせのテストが３回繰り返されているのは、GridSearchCVの仕様（内部で 3-fold クロスバリデーションを実行）による挙動です。

In [4]:
list_of_classifiers = []
for path in temp_path:
    classifier = fit_and_cross_validation(path)
    list_of_classifiers.append(classifier)

2017/04/05 PM 01:01:41 TrainingMessageFromCsv#__build_learning_training_messages count of learning data: 2156
2017/04/05 PM 01:01:41 TextArray#__init__ start


prepare_tf_idf_vectors: dataset=test_septeni_conversation.csv...


2017/04/05 PM 01:01:41 TextArray#to_vec start
2017/04/05 PM 01:01:41 TextArray#to_vec end
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:552: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=3.
  % (min_labels, self.n_folds)), Warning)


prepare_tf_idf_vectors: done in 0.798s.
GridSearchCV: search and fitting...
__accuracy_score: hidden_layer_sizes=(50,), accuracy=0.985879 (iterated=779, loss=0.058592)
__accuracy_score: hidden_layer_sizes=(50,), accuracy=0.983170 (iterated=716, loss=0.064008)
__accuracy_score: hidden_layer_sizes=(50,), accuracy=0.983508 (iterated=700, loss=0.056659)
__accuracy_score: hidden_layer_sizes=(100,), accuracy=0.987163 (iterated=603, loss=0.058811)
__accuracy_score: hidden_layer_sizes=(100,), accuracy=0.973352 (iterated=530, loss=0.067105)
__accuracy_score: hidden_layer_sizes=(100,), accuracy=0.982009 (iterated=559, loss=0.055140)
__accuracy_score: hidden_layer_sizes=(200,), accuracy=0.987163 (iterated=470, loss=0.061422)
__accuracy_score: hidden_layer_sizes=(200,), accuracy=0.971950 (iterated=417, loss=0.069171)
__accuracy_score: hidden_layer_sizes=(200,), accuracy=0.983508 (iterated=440, loss=0.057709)
__accuracy_score: hidden_layer_sizes=(50, 50), accuracy=0.988447 (iterated=764, loss=0.052

2017/04/05 PM 01:09:19 self.threshold: 0.5


GridSearchCV: done in 457.357s.
Evaluator: evaluating...


2017/04/05 PM 01:09:36 Evaluator#evaluate#elapsed time: 17244.342089 ms
2017/04/05 PM 01:09:36 accuracy: 0.972222222222


0.972222222222
Evaluator: done in 17.247s.


### (2-2) 選択されたモデルを確認

<b><a href="05.ipynb">デフォルトのグリッドサーチによる結果</a></b>とは、異なっていることが確認されました。

（プロダクションの __accuracy_score 関数 ではなく、MLPClassifier.score 関数が使用される影響と推測しています）

In [5]:
for classifier in list_of_classifiers:
    basename, X, y, vectorizer, estimator, evaluator = classifier
    params = estimator.get_params()

    print('[%s] best parameter: hidden_layer_sizes=%s, accuracy=%0.6f' % (
        basename, str(params['hidden_layer_sizes']), evaluator.accuracy
    ))

[test_septeni_conversation.csv] best parameter: hidden_layer_sizes=(50, 50), accuracy=0.972222
